# 112 feature

In [ ]:
import numpy as np
import scanpy as sc
import anndata as ad
import multimil as mtm
import numpy as np
import scanpy as sc
import scvi
import warnings

warnings.filterwarnings("ignore")

In [3]:
import datasets
import pandas as pd

meta_df = pd.read_csv("../hest_data/HEST_v1_1_0.csv", index_col=0)

In [5]:
meta_df_t = meta_df[meta_df['st_technology'] == 'Visium']
meta_df_t = meta_df_t[meta_df_t['species'] == 'Homo sapiens']

In [ ]:
meta_df_t['disease_state'].value_counts()

In [7]:
meta_df_t['disease_state'] = [str(i) for i in meta_df_t['disease_state'].values]
meta_df_t['oncotree_code'] = [str(i) for i in meta_df_t['oncotree_code'].values]

In [8]:
disease_update = []
for i in meta_df_t.index:
    if meta_df_t.loc[i]['oncotree_code'] != 'nan':
        disease_update.append(meta_df_t.loc[i]['oncotree_code'])
    else:
        disease_update.append(meta_df_t.loc[i]['disease_state'])

In [9]:
meta_df_t['disease_update'] = disease_update 

In [ ]:
set(meta_df_t['disease_update'] )

In [11]:
meta_df_t.index = [i for i in range(len(meta_df_t))]

In [12]:
meta_df_t_update = meta_df_t[meta_df_t['disease_update'].isin(['ACYC',
 'BLCA',
 'CESC',
 'COAD',
 'COADREAD',
 'EPM',
 'GBM',
 'HCC',
 'HGSOC',
 'Healthy',
 'IDC',
 'ILC',
 'LNET',
 'LUSC',
 'PAAD',
 'PRAD',
 'READ',
 'SCCRCC',
 'SOC',])]

In [ ]:
meta_df_t_update

In [14]:
import torch

In [14]:
import pickle
with open("./visium_gpfm_imagefeature.pickle", "rb") as f:
    input_data_visium = pickle.load(f)

In [15]:
input_data_new = []
for i in input_data_visium:
    input_data_new.append(i.mean(axis=0).numpy())

In [16]:
del input_data_visium

In [17]:
input_data_new = np.array(input_data_new)

In [ ]:
input_data_new.shape

In [19]:
adata = sc.read_h5ad("./visium_all.h5ad")

In [20]:
pseudo_list = []
for i in adata.obs['batch'].unique():
    pseudo_list.append(adata[adata.obs['batch'] == i].X.mean(axis=0))

In [21]:
train_data = np.array(pseudo_list)[:,0,:]

In [22]:
select_train_data = train_data[meta_df_t_update.index]

In [23]:
import sklearn.linear_model

In [24]:
import xgboost

In [26]:
import numpy as np
from sklearn.model_selection import KFold
result_list_new = []
X = meta_df_t_update.index


In [27]:
import sklearn.linear_model

In [ ]:
result_list_new = []
kf = KFold(n_splits=3, shuffle=True, random_state=2024)
for train, test in kf.split(X):
    train_batch = X[train]
    train_label = meta_df_t_update.loc[train_batch]['disease_update'].values


    model = sklearn.linear_model.LogisticRegression(n_jobs=-1)
    model.fit(np.concatenate((train_data[train_batch], input_data_new[train_batch]), axis=1), train_label)
    
    test_batch = X[test]
    test_label = meta_df_t_update.loc[test_batch]['disease_update'].values
    
    pred = model.predict(np.concatenate((train_data[test_batch], input_data_new[test_batch]), axis=1))
    
    dict_out = sklearn.metrics.classification_report(test_label,pred,digits=4, output_dict=True)
    single_out = [dict_out['accuracy'], dict_out['weighted avg']['precision'] , dict_out['weighted avg']['recall'] , dict_out['weighted avg']['f1-score']]
    result_list_new.append(single_out)

In [ ]:
for i in result_list_new:
    print(i[0], ' ', i[1], ' ', i[2], ' ', i[3])

In [ ]:
# If you want trying oversampling, please consider:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

result_list_new = []
kf = KFold(n_splits=3, shuffle=True, random_state=2024)
for train, test in kf.split(X):
    train_batch = X[train]
    train_label = meta_df_t_update.loc[train_batch]['disease_update'].values

    model = sklearn.linear_model.LogisticRegression(n_jobs=-1, max_iter=1000)
    
    ros = RandomOverSampler(random_state=42)
    
    X_train, y_train = ros.fit_resample(np.concatenate((train_data[train_batch], input_data_new[train_batch]), axis=1), train_label)
    
    model.fit(X_train, y_train)
    
    test_batch = X[test]
    test_label = meta_df_t_update.loc[test_batch]['disease_update'].values
    
    pred = model.predict(np.concatenate((train_data[test_batch], input_data_new[test_batch]), axis=1))
    
    dict_out = sklearn.metrics.classification_report(test_label,pred,digits=4, output_dict=True)
    single_out = [dict_out['accuracy'], dict_out['weighted avg']['precision'] , dict_out['weighted avg']['recall'] , dict_out['weighted avg']['f1-score']]
    result_list_new.append(single_out)

In [ ]:
#oversample plus cv
for i in result_list_new:
    print(i[0], ' ', i[1], ' ', i[2], ' ', i[3])

In [31]:
import sklearn.ensemble

In [ ]:
a = 1

In [46]:
def get_batch_id(data, sample_id):
    data_select = data[data['dataset_title'].isin(sample_id)]
    return data_select.index

In [47]:
import numpy as np
from sklearn.model_selection import KFold
result_list_new = []
X = meta_df_t_update['dataset_title'].unique()

In [48]:
select_train_data_new = np.array(pseudo_list)[:,0,:]

In [ ]:
select_train_data_new.shape

In [51]:
result_list_new = []
kf = KFold(n_splits=3, shuffle=True, random_state=2024)
for train, test in kf.split(X):
    train_data = X[train]
    train_index = get_batch_id(meta_df_t_update, train_data)
    train_label = meta_df_t_update.loc[train_index]['disease_update'].values
    
    enc = sklearn.preprocessing.LabelEncoder()
    train_label = enc.fit_transform(train_label)

    test_data = X[test]
    test_index = get_batch_id(meta_df_t_update, test_data)
    test_label = meta_df_t_update.loc[test_index]['disease_update'].values
    
    train_batch = train_index
    test_batch = test_index
    
    model = sklearn.linear_model.LogisticRegression(n_jobs=-1)
    
    model.fit(np.concatenate((select_train_data_new[train_batch], input_data_new[train_batch]), axis=1), train_label)
    
    pred = model.predict(np.concatenate((select_train_data_new[test_batch], input_data_new[test_batch]), axis=1))
    pred = enc.inverse_transform(pred)
    
    dict_out = sklearn.metrics.classification_report(test_label,pred,digits=4, output_dict=True)
    single_out = [dict_out['accuracy'], dict_out['weighted avg']['precision'] , dict_out['weighted avg']['recall'] , dict_out['weighted avg']['f1-score']]
    result_list_new.append(single_out)

In [ ]:
for i in result_list_new:
    print(i[0], ' ', i[1], ' ', i[2], ' ', i[3])